In [57]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
#import seaborn as sns

import random
from pprint import pprint

from sklearn.model_selection import train_test_split
from sklearn import metrics

In [58]:
df = pd.read_csv("data_banknote_authentication.txt")
df = df.rename(columns={"class": "label"})
df.head()

,variance_of_Wavelet_Transformed_image,skewness_of_Wavelet_Transformed_image,curtosis_of_Wavelet_Transformed_image,entropy_of_image,label
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


In [59]:
X = np.array(df[['variance_of_Wavelet_Transformed_image','skewness_of_Wavelet_Transformed_image','curtosis_of_Wavelet_Transformed_image','entropy_of_image']])
y = np.array(df['label'])
train_df, test_df, X_train, X_test, y_train, y_test = train_test_split(df, X, y, test_size=0.15, random_state=4)

In [60]:
def checkifpurity(data):
    
    label_column = data[:, -1]
    unique_classes = np.unique(label_column)

    if len(unique_classes) == 1:
        return True
    else:
        return False

In [61]:
def classifyingdata(data):
    
    label_column = data[:, -1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]
    
    return classification

In [62]:
def generate_potential_splits(data):
    
    potential_splits = {}
    _, n_columns = data.shape
    for column_index in range(n_columns - 1):          # excluding the last column which is the label
        values = data[:, column_index]
        unique_values = np.unique(values)
        
        type_of_feature = FEATURE_TYPES[column_index]
        if type_of_feature == "continuous":
            potential_splits[column_index] = []
            for index in range(len(unique_values)):
                if index != 0:
                    current_value = unique_values[index]
                    previous_value = unique_values[index - 1]
                    potential_split = (current_value + previous_value) / 2

                    potential_splits[column_index].append(potential_split)
        
        # feature is categorical
      
        elif len(unique_values) > 1:
            potential_splits[column_index] = unique_values
    
    return potential_splits

In [63]:
def split_data(data, split_column, split_value):
    
    split_column_values = data[:, split_column]

    type_of_feature = FEATURE_TYPES[split_column]
    if type_of_feature == "continuous":
        data_below = data[split_column_values <= split_value]
        data_above = data[split_column_values >  split_value]
    
    # feature is categorical   
    else:
        data_below = data[split_column_values == split_value]
        data_above = data[split_column_values != split_value]
    
    return data_below, data_above

In [64]:
def calculateingentropy(data):
    
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

In [65]:
def calculate_overall_entropy(data_below, data_above):
    
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n

    overall_entropy =  (p_data_below * calculateingentropy(data_below) 
                      + p_data_above * calculateingentropy(data_above))
    
    return overall_entropy

In [66]:
def determine_best_split(data, potential_splits):
    
    overall_entropy = 9999
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column=column_index, split_value=value)
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)

            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value

In [67]:
def determine_feature(df):
    
    feature_types = []
    n_unique_values_treshold = 15
    for feature in df.columns:
        if feature != "label":
            unique_values = df[feature].unique()
            example_value = unique_values[0]

            if (isinstance(example_value, str)) or (len(unique_values) <= n_unique_values_treshold):
                feature_types.append("categorical")
            else:
                feature_types.append("continuous")
    
    return feature_types

In [68]:
def decision_tree_algorithm(df, counter=0, min_samples=2, max_depth=5):
    
    # data preparations
    if counter == 0:
        global COLUMN_HEADERS, FEATURE_TYPES
        COLUMN_HEADERS = df.columns
        FEATURE_TYPES = determine_feature(df)
        data = df.values
    else:
        data = df           
    
    
    # base cases
    if (checkifpurity(data)) or (len(data) < min_samples) or (counter == max_depth):
        classification = classifyingdata(data)
        
        return classification

    
    # recursive part
    else:    
        counter += 1

        # helper functions 
        potential_splits = generate_potential_splits(data)
        split_column, split_value = determine_best_split(data, potential_splits)
        data_below, data_above = split_data(data, split_column, split_value)
        
        # determine question
        feature_name = COLUMN_HEADERS[split_column]
        type_of_feature = FEATURE_TYPES[split_column]
        if type_of_feature == "continuous":
            question = "{} <= {}".format(feature_name, split_value)
            
        # feature is categorical
        else:
            question = "{} = {}".format(feature_name, split_value)
        
        # instantiate sub-tree
        sub_tree = {question: []}
        
        # find answers (recursion)
        yes_answer = decision_tree_algorithm(data_below, counter, min_samples, max_depth)
        no_answer = decision_tree_algorithm(data_above, counter, min_samples, max_depth)
        
        # If the answers are the same, then there is no point in asking the qestion.
        # This could happen when the data is classified even though it is not pure
        # yet (min_samples or max_depth base case).
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree

In [69]:
decesiontree = decision_tree_algorithm(train_df, max_depth=4)
pprint(decesiontree)

{'variance_of_Wavelet_Transformed_image <= 0.760295': [{'skewness_of_Wavelet_Transformed_image <= 5.1608': [{'variance_of_Wavelet_Transformed_image <= 0.32016500000000003': [1.0,
                                                                                                                                                                              {'curtosis_of_Wavelet_Transformed_image <= 0.47013499999999997': [1.0,
                                                                                                                                                                                                                                                0.0]}]},
                                                                                                            {'variance_of_Wavelet_Transformed_image <= -3.4448999999999996': [{'entropy_of_image <= -1.915945': [1.0,
                                                                                                                 

In [70]:
def classifyingexample(example, tree):
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split(' ')

    # ask question
    if example[feature_name] <= float(value):
        answer = tree[question][0]
    else:
        answer = tree[question][1]

    # base case
    if not isinstance(answer, dict):
        return answer
    
    # recursive part
    else:
        residual_tree = answer
        return classifyingexample(example, residual_tree)

In [71]:
y_pred1 = test_df.apply(classifyingexample, axis=1, args=(tree,))
print("Accuracy:",metrics.accuracy_score(y_test, y_pred1))

Accuracy: 0.9514563106796117
